# Part One — Syntax and Style 


In the first part of your coursework, your task is to explore the syntax and style of a set of 19th Century novels using the methods and tools that you learned in class. The texts you need for this part are in the novels subdirectory of the texts directory in the coursework Moodle template. The texts are in plain text files, and the filenames include the title, author, and year of publication, separated by hyphens. 

The template code provided in PartOne.py includes function headers for some sub-parts of this ques- tion. The main method of your finished script should call each of these functions in order. To complete your coursework, complete these functions so that they perform the tasks specified in the questions below. You may (and in some cases should) define additional functions.

Re-assessment template 2025

Note: The template functions here and the dataframe format for structuring your solution is a suggested but not mandatory approach. You can use a different approach if you like, as long as you clearly answer the questions and communicate your answers clearly.


In [2]:
from nltk.tokenize import word_tokenize
# import nltk.word_tokenize # incorrect -- raises error see: https://www.nltk.org/api/nltk.tokenize.html
# import nltk
from nltk.corpus import cmudict
from nltk.tokenize import sent_tokenize

import spacy
from pathlib import Path

import pandas as pd
import os


nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2000000



## read_novels:
Each file in the novels directory contains the text of a novel, and the name of the file is the title, author, and year of publication of the novel, separated by hyphens. Complete the python function read_texts to do the following: 
- i. create a pandas dataframe with the following columns: text, title, author, year 
- ii. sort the dataframe by the year column before returning it, resetting or ignoring the dataframe index.

In [ ]:
def read_file(filename):
    with open(filename, 'r') as f:
        return f.read()

def read_novels(path=Path.cwd() / "zips" / "p1-texts" / "novels"):
    """Reads texts from a directory of .txt files and returns a DataFrame with the text, title,
    author, and year"""
    files = [file for file in os.listdir(path) if file[-4:] == '.txt']
    df_data = []

    for i in range(len(files)):
        ''' Create a dictionary, and add the file items to it, then use populated dictionary to create a dataframe '''
        file_dict = {}
        file_dict['text'] = read_file(os.path.join(path,files[i]))
        title, author, year = files[i].split('-')
        file_dict['title'] = title 
        file_dict['author'] = author
        file_dict['year'] = year[:-4]
        df_data.append(file_dict)
    df = pd.DataFrame(df_data)     
    df = df.sort_values('year') # return sorted by year (assumed default ascending)
    df.reset_index(inplace=True, drop=True)     
    return df     
    
df = read_novels()

In [4]:
df[0:2]

,text,title,author,year
0,\nCHAPTER 1\n\nThe family of Dashwood had long...,Sense_and_Sensibility,Austen,1811
1,'Wooed and married and a'.'\n'Edith!' said Mar...,North_and_South,Gaskell,1855


## nltk_ttr: 
This function should return a dictionary mapping the title of each novel to its type-token ratio. Tokenize the text using the NLTK library only. Do not include punctuation as tokens, and ignore case when counting types.

In [ ]:
def nltk_ttr(text):
    """Calculates the type-token ratio of a text. Text is tokenized using nltk.word_tokenize."""
    text_cleaned = [t.lower() for t in word_tokenize(text) if t.isalnum()] # I've assumed that we want the numbers, as well as text.
    vocab = set(text_cleaned) # dedupe words
    ttr = len(vocab) / len(text_cleaned) # ratio words to the text
    return ttr


def get_ttrs(df):
    """helper function to add ttr to a dataframe"""
    results = {}
    for i, row in df.iterrows():
        results[row["title"]] = nltk_ttr(row["text"])
    return results
get_ttrs(df)


## c) flesch_kincaid
This function should return a dictionary mapping the title of each novel to the Flesch-Kincaid reading grade level score of the text.  
Use the NLTK library for tokenization and the CMU pronouncing dictionary for estimating syllable counts.

[Readibility](https://readabilityformulas.com/learn-how-to-use-the-flesch-kincaid-grade-level/)
$$
{\displaystyle 0.39 * \left({\frac {\text{total words}}{\text{total sentences}}}\right)
    +11.8\left({\frac {\text{total syllables}}{\text{total words}}}\right)-15.59 }
$$


### `count_syl()`

In [3]:
import re
from nltk.corpus import cmudict
def count_syl(word, d):
    """Counts the number of syllables in a word given a dictionary of syllables per word.
    if the word is not in the dictionary, syllables are estimated by counting vowel clusters

    Args:
        word (str): The word to count syllables for.
        d (dict): A dictionary of syllables per word.

    Returns:
        int: The number of syllables in the word.
    """
    counts = 0
    pattern = r'[aeiouy]+'

    if word in d:
        word_cmu = d[word][0]
        counts += sum(char[-1].isdigit() for char in word_cmu)
    elif word not in d:
        vowel_words = re.findall(pattern, word.lower())
        counts += len(vowel_words)
    return counts

##### Test for `count_syl()`

In [ ]:
# text = df['text'][0]
# pattern = r'chapter [0-9] +'
# text = text.replace('\n', ' ').strip()
# text = re.sub(pattern, '', text.lower())
# d = cmudict.dict()
# sents=sent_tokenize(text.lower())
# words=[t.lower() for t in word_tokenize(text) if t.isalpha()]

# x = words[0:10] # sample of the words

# ''' Manual text of the function vs local use of the cmu dictionary '''

# temp=dict()
# for i in x:
#     counts=0
#     word_cmu = d[i][0]
#     counts += sum(char[-1].isdigit() for char in word_cmu)
#     temp[i] = (counts, word_cmu)
# print([t[0] for t in temp.values()] == [count_syl(i, d) for i in x])

# ''' And a test with some words not in the cmu dictionary'''
# y = ['aaaa', 'maaattaan']
# z = x+y
# print(list(zip(z,[count_syl(i, d) for i in z])))
# print(sum([count_syl(word, d) for word in z]))

True
[('the', 1), ('family', 3), ('of', 1), ('dashwood', 2), ('had', 1), ('long', 1), ('been', 1), ('settled', 2), ('in', 1), ('sussex', 2), ('aaaa', 1), ('maaattaan', 2)]


### `fk_level()`, `get_fks()`, `add_fks_to_df()`

In [ ]:

import nltk 
def fk_level(text, d):
    """Returns the Flesch-Kincaid Grade Level of a text (higher grade is more difficult).
    Requires a dictionary of syllables per word.

    Args:
        text (str): The text to analyze.
        d (dict): A dictionary of syllables per word.

    Returns:
        float: The Flesch-Kincaid Grade Level of the text. (higher grade is more difficult)
    """
    text = text.replace('\n', ' ').strip() #* And removing line breaks "\n"
    
    pattern = r'chapter [0-9] +' #* adding this so that chapter breaks are not counted
    text = re.sub(pattern, '', text.lower())
    
    sents=sent_tokenize(text.lower())
    words=[t.lower() for t in word_tokenize(text) if t.isalpha()]
    temp = dict()
    total_syllables = sum([count_syl(word, d) for word in words])
    fkl = 0.39 * (len(words) / len(sents)) + 11.8 * (total_syllables/ len(words))- 15.59
    # print(total_syllables, len(words), len(sents), fkl)

    return fkl


def get_fks(df):
    """helper function to add fk scores to a dataframe"""
    results = {}
    cmudict = nltk.corpus.cmudict.dict()
    for i, row in df.iterrows():
        results[row["title"]] = round(fk_level(row["text"], cmudict), 4)
    return results

def add_fks_to_df(df):
    ''' function to add scores to the dataframe, as referenced in get_fks() '''
    fks = get_fks(df)
    temp = pd.DataFrame({'title_f': fks.keys(), 'fks':fks.values()})
    df = df.merge(temp, left_on='title', right_on='title_f')
    return df.drop(columns='title_f')
df = add_fks_to_df(df)


## d) flesch_kincaid **todo**
When is the Flesch Kincaid score *not* a valid, robust or reliable estimator of text difficulty? Give two conditions. (Text answer, 200 words maximum).

## e) parse
The goal of this function is to process the texts with spaCy’s tokenizer and parser, and store the processed texts. Your completed function should: 
- i. Use the spaCy nlp method to add a new column to the dataframe that contains parsed and tokenized Doc objects for each text. 
- ii. Serialise the resulting dataframe (i.e., write it out to disk) using the pickle format. 
- iii. Return the dataframe. 
- iv. Load the dataframe from the pickle file and use it for the remainder of this coursework part. Note: one or more of the texts may exceed the default maximum length for spaCy’s model. You will need to either increase this length or parse the text in sections.

In [3]:

def parse(df, store_path=Path.cwd() / "pickles", out_name="parsed.pickle"):
    """Parses the text of a DataFrame using spaCy, stores the parsed docs as a column and writes the resulting  DataFrame to a pickle file"""
    docs = {}
    for i in range(len(df)):
        docs[i] = nlp(df['text'][i]) # create spacy docs for each cell "text"
        # print(f"Document {i+1} / {len(df)} parsed.") #? added to see process when parsing (takes a while)
    df['docs'] = docs.values()
    df.to_pickle(os.path.join(store_path,out_name)) #~ write to pickle
    # return df #^ To check the returned df
    
def load_from_pickle(store_path=Path.cwd() / "pickles", out_name="parsed.pickle"):
    df = pd.read_pickle(os.path.join(store_path, out_name))
    return df

# df = parse(df) #^ To check the returned df with spacy doc objects

''' Load from pickle '''
df = load_from_pickle() 

#? To check the returned type
# type(df['docs'][0]) # ~spacy.tokens.doc.Doc 

## f) Working with parses: **todo**
the final lines of the code template contain three `for` loops. Write the functions needed to complete these loops so that they print: 

### i. The title of each novel and a list of the ten most common syntactic objects overall in the text. 

In [ ]:
from collections import Counter 

def extract_top_n_pos(doc, n, output_type, with_punct):
    if with_punct:
        pos_counts = Counter([token.pos_ for token in doc])
    else: # * without punctuation
        pos_counts = Counter([token.pos_ for token in doc if token.pos_ != 'PUNCT'])
    if output_type == 'list':
        return [pos for pos, count in pos_counts.most_common()][:n]
    elif output_type == 'pairs':
        return pos_counts.most_common()[:n]

def print_title_top_pos(df, n=10, output_type = 'list', with_punct=True):
    top_dict = dict()
    for i in range(len(df)):
        n_pos = extract_top_n_pos(df['docs'][i], n, output_type, with_punct)
        top_dict[df['title'][i]] = n_pos
    for key, value in top_dict.items():
        print(f"{key}: {value}")
    
# print_title_top_pos(df) # * default - title: list of top POS (including punctuation)
print_title_top_pos(df, output_type='pairs', with_punct=False) # * call to show POS type and count value (tuples), excluding punctuation

Sense_and_Sensibility: [('PRON', 18596), ('NOUN', 17361), ('VERB', 14896), ('ADP', 14146), ('AUX', 10153), ('DET', 9076), ('ADV', 8282), ('ADJ', 7880), ('PROPN', 5847), ('CCONJ', 4753)]
North_and_South: [('PRON', 28772), ('NOUN', 28271), ('VERB', 25921), ('ADP', 20353), ('AUX', 14662), ('DET', 14013), ('ADJ', 12385), ('ADV', 11246), ('PROPN', 8897), ('CCONJ', 7741)]
A_Tale_of_Two_Cities: [('NOUN', 23904), ('PRON', 18912), ('VERB', 17859), ('ADP', 17102), ('DET', 13761), ('SPACE', 12289), ('ADJ', 9264), ('AUX', 8885), ('ADV', 7316), ('PROPN', 6173)]
Erewhon: [('NOUN', 13715), ('PRON', 11292), ('VERB', 10127), ('ADP', 9573), ('DET', 7923), ('AUX', 7020), ('ADJ', 5995), ('ADV', 5343), ('CCONJ', 3913), ('SCONJ', 3355)]
The_American: [('PRON', 24225), ('VERB', 20253), ('NOUN', 19413), ('ADP', 13555), ('SPACE', 11755), ('DET', 10650), ('AUX', 10418), ('ADJ', 9554), ('ADV', 7830), ('PROPN', 6938)]
Dorian_Gray: [('PRON', 13483), ('NOUN', 12681), ('SPACE', 11301), ('VERB', 11241), ('ADP', 8663)

- ii. The title of each novel and a list of the ten most common syntactic subjects of the verb ‘to hear’ (in any tense) in the text, ordered by their frequency. 
- iii. The title of each novel and a list of the ten most common syntactic subjects of the verb ‘to hear’ (in any tense) in the text, ordered by their Pointwise Mutual Information.

In [ ]:


def adjective_counts(doc):
    """Extracts the most common adjectives in a parsed document. Returns a list of tuples."""
    pass




def subjects_by_verb_pmi(doc, target_verb):
    """Extracts the most common subjects of a given verb in a parsed document. Returns a list."""
    pass
    for i, row in df.iterrows():
        print(row["title"])
        print(subjects_by_verb_pmi(row["parsed"], "hear"))
        print("\n")



def subjects_by_verb_count(doc, verb):
    """Extracts the most common subjects of a given verb in a parsed document. Returns a list."""
    pass
    for i, row in df.iterrows():
        print(row["title"])
        print(subjects_by_verb_count(row["parsed"], "hear"))
        print("\n")








## g) Ten marks are allocated for your github commit history. 
You should make regular, atomic commits with concise but informative commit messages. See the section titled Submission (both questions) for more details.

In [ ]:
if __name__ == "__main__":
    """
    uncomment the following lines to run the functions once you have completed them
    """
    #path = Path.cwd() / "p1-texts" / "novels"
    #print(path)
    #df = read_novels(path) # this line will fail until you have completed the read_novels function above.
    #print(df.head())
    #nltk.download("cmudict")
    #parse(df)
    #print(df.head())
    #print(get_ttrs(df))
    #print(get_fks(df))
    #df = pd.read_pickle(Path.cwd() / "pickles" /"name.pickle")
    # print(adjective_counts(df))
    """ 
    for i, row in df.iterrows():
        print(row["title"])
        print(subjects_by_verb_count(row["parsed"], "hear"))
        print("\n")

    for i, row in df.iterrows():
        print(row["title"])
        print(subjects_by_verb_pmi(row["parsed"], "hear"))
        print("\n")
    """

# Part Two — Feature Extraction and Classification 
In the second part of the coursework, your task is to train and test machine learning classifiers on a dataset of political speeches. The objective is to learn to predict the political party from the text of the speech. The texts you need for this part are in the speeches sub-directory of the texts directory of the coursework Moodle template. For this part, you can structure your python functions in any way that you like, but pay attention to exactly what information (if any) you are asked to print out in each part. Your final script should print out the answers to each part where required, and nothing else.

## a) Read the hansard40000.csv dataset in the texts directory into a dataframe. 
Sub- set and rename the dataframe as follows: 
- i. rename the ‘Labour (Co-op)’ value in ‘party’ column to ‘Labour’, and then: 
- ii. remove any rows where the value of the ‘party’ column is not one of the four most common party names, and remove the ‘Speaker’ value. 
- iii. remove any rows where the value in the ‘speech_class’ column is not ‘Speech’. 
- iv. remove any rows where the text in the ‘speech’ column is less than 1000 characters long
- Print the dimensions of the resulting dataframe using the shape method.

Assuming we ignore blanks (`NaN`)

`df['speech_class'].value_counts(dropna=False)`
```
party
Conservative                        25079
Labour                               8038
Scottish National Party              2303
NaN                                  1647
.....

`df['speech_class'].value_counts()`
The top 4n less "Speaker"
party
Conservative                        25079
Labour                               8038
Scottish National Party              2303
Liberal Democrat                      803
.....
```

In [ ]:
import pandas as pd
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def csv_to_df(path=Path.cwd() / "zips" / "p2-texts"):
    df = pd.read_csv(os.path.join(path,os.listdir(path)[0]))

    #^ - i. rename the ‘Labour (Co-op)’ value in ‘party’ column to ‘Labour’, and then: 
    df['party'].replace("Labour (Co-op)", "Labour", inplace=True)


    #^ - ii. remove any rows where the value of the ‘party’ column is not one of the four most common party names, and remove the ‘Speaker’ value. 
    values = ['Conservative', 'Labour', 'Scottish National Party', 'Liberal Democrat',]
    df = df[df['party'].isin(values)]

    #^ - iii. remove any rows where the value in the ‘speech_class’ column is not ‘Speech’. 
    df = df[df['speech_class'] == 'Speech']

    df.reset_index(drop=True, inplace=True) #? NB: needs reseting, otherwise the next step doesn't work


    #^ - iv. remove any rows where the text in the ‘speech’ column is less than 1000 characters long
    # indices= []
    ind= []
    for i, s in enumerate(df['speech'].values):
        # print(len(s))
        if len(s) < 1000:
            # indices.append((i, len(s)))
            ind.append(i)

    df = df[~df.index.isin(ind)]

    #^ - Print the dimensions of the resulting dataframe using the shape method.
    # len(df) == 36223-28139
    print(df.shape)
    return df #? Although this might not be wanted? "Function to print and nothing else?"

In [ ]:
df = csv_to_df()

## b) Vectorise the speeches using TfidfVectorizer from scikit-learn. 
Use the default parameters, except for omitting English stopwords and setting max_features to 3000. Split the data into a train and test set, using stratified sampling, with a random seed of 26.

## c) Random forest and SVM
Train RandomForest (with n_estimators=300) and SVM with linear kernel classifiers on the training set, and print the scikit-learn macro-average f1 score and classification report for each classifier on the test set. The label that you are trying to predict is the ‘party’ value. 

## d) N-Grams
Adjust the parameters of the Tfidfvectorizer so that unigrams, bi-grams and tri-grams will be considered as features, limiting the total number of features to 3000. Print the classification report as in 2(c) again using these parameters. 

## e) Implement a new custom tokenizer and pass it to the tokenizer argument of Tfidfvectorizer. 
You can use this function in any way you like to try to achieve the best classification performance while keeping the number of features to no more than 3000, and using the same three classifiers as above. Print the clas- sification report for the best performing classifier using your tokenizer. Marks will be awarded both for a high overall classification performance, and a good trade-off between classification performance and efficiency (i.e., using fewer parameters).

## f) Explain your tokenizer function and discuss its performance. 

## g) Githistory
Ten marks are allocated for your github commit history. You should make regular, atomic commits with concise but informative commit messages. See the section below titled Submission (both questions) for more details. Part Two total marks: 50

# Git requirements

Submit using github classroom and confirm on Moodle The code template for this coursework part is made available to you as a Git repository on GitHub, via an invitation link for GitHub Classroom. 

- 1. First you follow the invitation link for the coursework that is available on the Moodle page of the module. 
- 2. Then clone the Git repository from the GitHub server that GitHub will create for you. Initially it will contain README.md and a folder structure for Part One and Part Two with placeholder python scripts (you can change these to Jupyter notebook files if you prefer). 
- 3. Enter your name in README.md (this makes it easy for us to see whose code we are marking) 
- 4. You must also enter the following Academic Declaration into README.md for your submission:
> “I have read and understood the sections of plagiarism in the College Policy on assessment offences and confirm that the work is my own, with the work of others clearly acknowledged. I give my permission to submit my report to the plagiarism testing database that the College is using and test it using plagiarism detection software, search engines or meta-searching software.” 
This refers to the document at: [link](https://www.bbk.ac.uk/student-services/exams/plagiarism-guidelines)

5. Whenever you have made a change that can “stand on its own”, say, “Implemented tokenizer method”, this is a good opportunity to commit the change to your local repository and also to push your changed local repository to the GitHub server. As a rule of thumb, in collaborative software development it is common to require that the code base should at least still compile after each commit.

- Entering your name in README.md (using a text editor), then doing a commit of your change to the file into the local repository, and finally doing a push of your local repository to the GitHub server would be an excellent way to start your coursework activities. 
- You can benefit from the GitHub server also to synchronise between, e.g., the Birkbeck lab machines and your own computer. You push the state of your local repository in the lab to the GitHub server before you go home; later, you can pull your changes to the repository on your home computer (and vice versa). Use meaningful commit messages (e.g., “Implemented the pickle output for Q1(e)”, or “fixed a bug in the PMI calculation”), and do not forget to push your changes to the GitHub server! 
- For marking, we plan to clone your repositories from the GitHub server shortly after the submission deadline. We additionally require you to confirm your github submission via a ‘confirm submission’ form on Moodle. The time of this confirmation will tell us if you would like your code to be considered for the regular (uncapped) deadline or for the late (capped) deadline two weeks later.

Deadlines 
-  The submission deadline is: 26th June 2025, 14:00 UK time. 
- The late cut-off deadline for receiving a capped mark is: 10th July 2025, 14:00 UK time.